In [6]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [23]:
''' Helper functions '''


def append_from_csv(file: Path, sep: str) -> None:
    try:
        df: pd.DataFrame = pd.read_csv(file, sep=sep, dtype=str)
    except Exception as ex:
        print(f'{file} :')
        print(ex)
    else:
        if df.shape[1] == 5:
            try:
                df = df.applymap(lambda x: str(x).replace(',', '.'))
                df['gFx'] = df['gFx'].astype(float)
                df['gFy'] = df['gFy'].astype(float)
                df['gFz'] = df['gFz'].astype(float)
            except Exception as ex:
                print(f'{file} :')
                print(ex)
            else:
                DataFrames.append(df)
                Types.append(correct_type)
                
                
def process_frame(df: pd.DataFrame, label: int) -> None:
    start = int(0.2 * len(df))
    end = -int(0.2 * len(df))

#     x_mean = np.mean(df.gFx[start:end])
#     y_mean = np.mean(df.gFy[start:end])
#     z_mean = np.mean(df.gFz[start:end])
    
#     return pd.Series(
#         [x_mean, y_mean, z_mean, label],
#         index=['x_mean', 'y_mean', 'z_mean', 'label']
#     )
    
    accel = np.mean((df.gFx[start:end] ** 2 + df.gFy[start:end] ** 2 + df.gFz[start:end] ** 2) ** 0.5)
    
    return pd.Series(
        [accel, label],
        index=['accel', 'label']
    )

In [3]:
''' Сбор данных из файлов c разметкой типов '''


type_patterns = {
    ('cтояние', 'тояни', 'месте'): {'name': 'стояние', 'id': 0},
    ('ходьба', 'одьб', 'аг'): {'name': 'ходьба', 'id': 1},
    ('бег'): {'name': 'бег', 'id': 2},
    ('велосипед', 'елоси'): {'name': 'велосипед', 'id': 3},
    ('лестница', 'естн', 'одъ'): {'name': 'лестница', 'id': 4},
    ('автомобиль', 'втомо', 'ашин'): {'name': 'автомобиль', 'id': 5},
    ('метро'): {'name': 'метро', 'id': 6},
    ('автобус', 'автоб'): {'name': 'автобус', 'id': 7},
    ('самокат', 'амок'): {'name': 'самокат', 'id': 8}
}


DataFrames: list[pd.DataFrame] = []
Types: list[int] = []
public_data_path = Path('./public_data')


for sub_path in public_data_path.iterdir():
        
    if sub_path.is_file():

        try:
            type_str = str(sub_path.name).split('_')[-2]
        except IndexError:
            continue
        else:
            correct_type = int()
            
            for key, value in type_patterns.items():
                if type_str.lower() in key:
                    correct_type = value['id']
                    
            if correct_type not in range(9):
                raise ValueError(f'Ошибка при определении типа движения. {type_str = }')

        append_from_csv(sub_path, ',')
        append_from_csv(sub_path, ';')

public_data/dimiii - буба_велосипед_1.csv :
Error tokenizing data. C error: Expected 5 fields in line 103530, saw 8

public_data/Dima Sokolov - Соколов_Бег_2.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Бег_2.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Бег_3.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Бег_3.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Бег_4.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Бег_4.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_1.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_1.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_3.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_3.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_4.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Стояние_4.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Ходьба_1.csv.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Ходьба_1.csv.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Ходьба_2.csv :
'gFx'
public_data/Dima Sokolov - Соколов_Ходьба_2.csv

In [24]:
train = pd.DataFrame(columns=['accel', 'label'])


for frame, label in zip(DataFrames, Types):
    train = train.append(process_frame(frame, label), ignore_index=True)

In [30]:
X = np.array(train.accel).reshape(-1, 1)
y = train.label

model = LogisticRegression(random_state=0).fit(X, y)

In [31]:
model.score(X, y)

0.4782608695652174